In [8]:
import os
import time
import tqdm

import numpy as np
import pandas as pd
import audb
import audiofile
from scipy.io.wavfile import read as read_wav
from matplotlib import pyplot as plt
import opensmile

import sklearn 
from sklearn.model_selection import train_test_split #train/test split 
from sklearn.svm import SVC #Model
from sklearn.metrics import confusion_matrix #Metrics 
from sklearn.metrics import f1_score 
from sklearn.model_selection import cross_val_score #Cross-validation
from sklearn.model_selection import KFold

import scipy
import random #for shuffling values 
from scipy import stats

#List of prosodic features we will observe

features_prosody= ['Loudness_sma3', 
                   'hammarbergIndex_sma3',
                   'F0semitoneFrom27.5Hz_sma3nz',
                    'slope0-500_sma3',
                  'slope500-1500_sma3',
                   'jitterLocal_sma3nz',
 'shimmerLocaldB_sma3nz',
 'HNRdBACF_sma3nz',
 'mfcc1_sma3',
 'mfcc2_sma3',
 'mfcc3_sma3',
 'mfcc4_sma3']

# Function to perform one-hot encoding
def encode(label):
    if 'non-hate' in label:
        return 0
    elif 'hate' in label:
        return 1

#Gets functional features of all audios 
def lowlevel_features(list_with_files):
    path= "data/clean_audio1/"
    
    #  Define feature extractor
    smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
    )

    df_data= []

    for i in (list_with_files):
        f= path + i
        sampling_rate, data = read_wav(f) #gets sampling rate and dimensions of data
        #print(f"Sampling rate of {f}:{sampling_rate}")
        
        db = audb.load('emodb',
        version='1.1.1',
        format='wav',
        mixdown=True,
        sampling_rate=sampling_rate,
        media='wav/03a01.*',  # load subset
        full_path=False,
        verbose=False,)

        #extract features
        signal, sampling_rate = audiofile.read(f, duration=120, always_2d=True)
        data= smile.process_signal(signal, sampling_rate)
        df_data.append(data.loc[:,features_prosody].values)

   # df= pd.concat(df_data, ignore_index=True)
    return df_data
        
def t_test(dataframe):
    for i in features_prosody:
        class_0= dataframe[i][dataframe['label']==0].values
        class_1= dataframe[i][dataframe['label']== 1].values 
        t_statistic, p_value = stats.ttest_ind(class_0, class_1)
        print(f"T-statistic {i}:", t_statistic)
        print(f"P-value {i}:", p_value)
    

In [6]:
df=pd.read_csv("D:/CCiL/TFM/model/data/data_spanish_dataset.csv")

# Encoding categorical variables. Not hate speech = 0. Hate speech = 1. 
df['label'] = df['label'].apply(lambda x: encode(x))

#Storing labels in array 
labels= df['label'].values
print(labels)

[0 1 0 1 1 0 0 1 0 1 0 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 0 0 0 1 1 0 1 0 1 1 1 1 0 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1
 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 0 1]


In [9]:
#process wav files 
path="data/clean_audio1"
dir_list = os.listdir(path)
feature_data= lowlevel_features(dir_list)

In [11]:
# Find the shape of the largest matrix
max_shape = max(matrix.shape for matrix in feature_data)

# Zero-pad the rows of matrices
padded_matrices = []
for matrix in feature_data:
    pad_width = ((0, max_shape[0] - matrix.shape[0]), (0, 0))
    padded_matrix = np.pad(matrix, pad_width, mode='constant', constant_values=0)
    padded_matrices.append(padded_matrix)


In [12]:
# Check the shape of padded matrices
for matrix in padded_matrices:
    print(matrix.shape)

(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(119